<a href="https://colab.research.google.com/github/normalnon/Advanced-Data-Filtering/blob/main/Workshop4_Cleanpipline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. ติดตั้ง Libraries
!pip install pythainlp transformers torch rapidfuzz beautifulsoup4 pandas emoji
!pip install epitran

import os
import re
import string
import random
import hashlib
import json
import logging
import warnings
import unicodedata
import emoji
from functools import lru_cache
from typing import List, Dict, Tuple, Optional, Sequence, Set
from collections import defaultdict
from dataclasses import dataclass

# สร้างไฟล์ Mock Gazetteer สำหรับทดสอบ Address Cleaner (เพื่อป้องกัน Error file not found)
mock_gazetteer = {
    "bangkok": {
        "กรุงเทพมหานคร-province": [
            "พญาไท-district", "ดินแดง-district", "ปทุมวัน-district",
            "สามเสนใน-subdistrict", "ลุมพินี-subdistrict"
        ],
        "เชียงใหม่-province": ["เมือง-district", "สุเทพ-subdistrict"]
    }
}
with open('th_gazetteer.json', 'w', encoding='utf-8') as f:
    json.dump(mock_gazetteer, f, ensure_ascii=False)

print("Installation and Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10744 sha256=195248e156aa8b08904187e531cee4f5c5ae97827e2759710b6d0fe3b4aef163
  Stored in directory: /root/.cache/pip/wheels/f2/67/7d/2e80818c2f3dc8f0735d0810338c47e95d3212114ab97b4ede
Successfully built unicodecsv
Installation and Setup Complete.


In [ ]:
# @title 2. ระบบ Regex สำหรับกรองสแปม คำหยาบ และทำความสะอาดข้อความ (Escape Backslash Version)

import re
import pandas as pd

class ContentRegexFilter:
    def __init__(self):
        # --- 1. Detection Regex (Spam/Rude/Contact) ---

        # ========== 1.1 Gambling Pattern ==========
        gambling_keywords = [
            # ไทย
            "เว็บพนัน", "เว็บ\\s*พนัน", "พนันออนไลน์", "สล็อต", "สล๊อต", "สลอต", "sl0t",
            "บาคาร่า", "บาคารา", "bacara", "bacarat",
            "คาสิโน", "casin0", "casino",
            "แทงบอล", "บอลสเต็ป", "บอล\\s*สเต็ป", "แทงบอลชุด",
            "หวย", "หวยออนไลน์", "หวยรัฐบาล", "ลอตเตอรี่", "lotto", "lottery", "แทงหวย",
            "ยิงปลา", "เกมยิงปลา", "ป๊อกเด้ง", "ไฮโล", "รูเล็ต", "roulette",
            "เดิมพัน", "เสี่ยงโชค", "แจกทุน", "เครดิตฟรี", "free\\s*credit",
            "ฝากถอน", "ฝาก\\s*ถอน", "วอเลท", "wallet", "truewallet", "ทรูวอเลท",
            "แจ็คพอต", "jackpot", "spin", "แตกง่าย", "ถอนไม่อั้น", "เว็บตรง", "ทางเข้าเล่น",

            # อังกฤษ / ชื่อเว็บ
            "pg", "pgslot", "p\\s*g\\s*slot", "bet", "bett?ing", "gclub", "ufabet", "joker",
            "xo", "918kiss", "sagaming", "sa\\s*gaming", "sexxy"
        ]

        self.gambling_pattern = re.compile(
            "(" + "|".join(gambling_keywords) + ")",
            re.IGNORECASE
        )

        # ========== 1.2 Rude Words Pattern ==========
        rude_words_list = [
            "กู","มึง","สัส","สาด","เหี้ย","เฮี้ย","เหีย","เชี่ย","เชี้ย",
            "แม่ง","พ่อง","พ่อมึง","แม่มึง",
            "สันดาน","ชาติหมา","ชิบหาย","ฉิบหาย","ห่า",
            "โง่","ปัญญาอ่อน","ตอแหล","ระยำ","สถุล","สารเลว","ดอกทอง",
            "จังไร","อัปรีย์","อีดอก","เสือก",
            "fuck","f*ck","shit","bitch","asshole","damn"
        ]

        self.rude_pattern = re.compile(
            "(" + "|".join(map(re.escape, rude_words_list)) + ")",
            re.IGNORECASE
        )

        # ========== 1.3 Fuzzy Obscene Pattern ==========
        self.fuzzy_obscene_pattern = re.compile(
            "(ค[\\W_]*ย|ค[\\W_]*ว[\\W_]*ย|"
            "เ[\\W_]*ห[\\W_]*ี้[\\W_]*ย|เ[\\W_]*ช[\\W_]*ี[\\W_]*ย|"
            "ส[\\W_]*ั[\\W_]*ส|ส[\\W_]*า[\\W_]*ด|"
            "ห[\\W_]*ี|แ[\\W_]*ต[\\W_]*ด|"
            "เ[\\W_]*ย[\\W_]*็[\\W_]*ด)",
            re.IGNORECASE
        )

        # ========== 1.4 Contact ==========
        self.email_pattern = re.compile("[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}")
        self.phone_pattern = re.compile("0[0-9]{1,2}[-\\s]?[0-9]{3}[-\\s]?[0-9]{3,4}")

        # --- 2. Cleaning Regex (Repetitive) ---
        self.repeat_word_pattern = re.compile("(\\b\\S+\\b)(?:\\s+\\1){3,}", re.IGNORECASE)
        self.repeat_line_pattern = re.compile("(^.+$\\n)(?:\\1){3,}", re.MULTILINE)

        # --- 3. Header/Footer Patterns ---
        self.header_footer_patterns = [
            "^\\s*(?:page|หน้า)\\s*\\d+(?:\\s*[/of]\\s*\\d+)?\\s*$",
            "^\\s*\\d+\\s*[/]\\s*\\d+\\s*$",
            "^\\s*(?:confidential|ลับ|เอกสารราชการ)\\s*$",
            "^\\s*\\d{1,2}[/-]\\d{1,2}[/-]\\d{2,4}\\s*$"
        ]

    # --- Analysis Methods ---
    def check_gambling(self, text):
        found = self.gambling_pattern.findall(text)
        return bool(found), list(set(found))

    def check_rude_words(self, text):
        found1 = self.rude_pattern.findall(text)
        found2 = self.fuzzy_obscene_pattern.findall(text)
        found = list(set(found1 + found2))
        return bool(found), found

    def extract_email(self, text):
        return self.email_pattern.findall(text)

    def extract_phone(self, text):
        return self.phone_pattern.findall(text)

    # --- Cleaning Methods ---
    def remove_repetitive(self, text):
        text = self.repeat_line_pattern.sub("\\1", text)
        text = self.repeat_word_pattern.sub("\\1", text)
        return text

    def remove_header_footer(self, text, threshold_lines=3):
        lines = text.strip().split("\n")
        if len(lines) <= threshold_lines * 2:
            return text

        def is_header_footer(line):
            line = line.strip()
            if not line: return False
            for pat in self.header_footer_patterns:
                if re.match(pat, line, re.IGNORECASE):
                    return True
            if self.email_pattern.fullmatch(line) or line.startswith("http"):
                return True
            return False

        start_idx = 0
        for i in range(min(threshold_lines, len(lines))):
            if is_header_footer(lines[i]):
                start_idx = i + 1
            else:
                break

        end_idx = len(lines)
        for i in range(len(lines) - 1, len(lines) - 1 - threshold_lines, -1):
            if i < start_idx: break
            if is_header_footer(lines[i]):
                end_idx = i
            else:
                break

        return "\n".join(lines[start_idx:end_idx])

    def clean_and_analyze(self, text):
        cleaned_text = self.remove_header_footer(text)
        cleaned_text = self.remove_repetitive(cleaned_text)

        is_gambling, gambling_words = self.check_gambling(cleaned_text)
        is_rude, rude_words = self.check_rude_words(cleaned_text)
        emails = self.extract_email(cleaned_text)
        phones = self.extract_phone(cleaned_text)

        return {
            "original_text": text,
            "cleaned_text": cleaned_text,
            "is_gambling": is_gambling,
            "gambling_keywords": gambling_words,
            "is_rude": is_rude,
            "rude_keywords": rude_words,
            "emails_found": emails,
            "phones_found": phones
        }


In [ ]:
# --- Test Logic ---
regex_processor = ContentRegexFilter()

test_cases = [
    # ====== GAMBLING: ตรง ๆ ======
    "สนใจเล่นบาคาร่า เว็บตรง ไม่ผ่านเอเย่นต์ แจกเครดิตฟรี",
    "เล่นสล็อต pgslot แตกง่าย ถอนไม่อั้น",
    "เว็บพนัน ฝากถอน ผ่านวอเลท",
    "แทงหวย หวยออนไลน์ ลอตเตอรี่",
    "casino slot bet jackpot",

    # ====== GAMBLING: แปลงรูป / เว้นวรรค ======
    "p g s l o t แจกเครดิตฟรี",
    "sl0t แตกหนักมาก",
    "c a s i n o online",
    "บ า ค า ร่า เล่นง่าย",
    "เ ว็ บ พ นั น ดี ที่ สุด",

    # ====== GAMBLING: ชื่อค่าย ======
    "ufabet แจกโบนัส",
    "gclub ทางเข้า",
    "joker gaming",
    "918kiss",
    "sa gaming sexxy",

    # ====== RUDE: ตรง ๆ ======
    "มึงมันโง่จริงๆ",
    "ไอ้สัส เหี้ยเอ้ย",
    "แม่งโคตรระยำ",
    "พ่อมึง แม่มึง",
    "หน้าหี ชิบหาย",

    # ====== RUDE: แปลงรูป ======
    "เ หี้ ย จริงๆ",
    "สั ส มาก",
    "เ ชี่ ย อะไรเนี่ย",

    # ====== MIX: พนัน + หยาบ ======
    "เว็บพนันเหี้ยๆ แบบนี้ ใครจะไปเล่น",
    "sl0t โคตรสัส แตกก็ไม่แตก",

    # ====== CONTACT ======
    "ติดต่อเราได้ที่ support@casino.com",
    "อีเมล: test.user123@gmail.com",
    "โทร 081-234-5678",
    "โทร 0891234567",

    # ====== HEADER / FOOTER ======
    "Page 1 of 5\nนี่คือเนื้อหา\nข้างใน",
    "หน้า 2/10\nรายงานผล\nจบ",
    "01/01/2025\nเนื้อหาจริง\n02/01/2025",
    "CONFIDENTIAL\nข้อมูลภายใน\nCONFIDENTIAL",

    # ====== REPETITION ======
    "ฟรี ฟรี ฟรี ฟรี ฟรี มากๆ",
    "ดีมาก\nดีมาก\nดีมาก\nดีมาก\nดีมาก\nเนื้อหา",
    "wow wow wow wow wow wow",

    # ====== CLEAN / NORMAL TEXT ======
    "นี่คือเอกสารราชการเกี่ยวกับการขออนุญาตก่อสร้าง",
    "ระบบนี้ใช้สำหรับวิเคราะห์ข้อมูลนักศึกษา",
    "กรุณาติดต่อฝ่ายทะเบียน",
    "วันนี้อากาศดีมาก เหมาะกับการทำงาน"
]

print("--- Testing FULL Coverage ---")
results = []
for t in test_cases:
    res = regex_processor.clean_and_analyze(t)
    results.append(res)

df_res = pd.DataFrame(results)

display(df_res[[
    "cleaned_text",
    "is_gambling",
    "gambling_keywords",
    "is_rude",
    "rude_keywords",
    "emails_found",
    "phones_found"
]])


--- Testing FULL Coverage ---


,cleaned_text,is_gambling,gambling_keywords,is_rude,rude_keywords,emails_found,phones_found
0,สนใจเล่นบาคาร่า เว็บตรง ไม่ผ่านเอเย่นต์ แจกเคร...,True,"[บาคาร่า, เว็บตรง, เครดิตฟรี]",False,[],[],[]
1,เล่นสล็อต pgslot แตกง่าย ถอนไม่อั้น,True,"[สล็อต, pg, แตกง่าย, ถอนไม่อั้น]",False,[],[],[]
2,เว็บพนัน ฝากถอน ผ่านวอเลท,True,"[วอเลท, เว็บพนัน, ฝากถอน]",False,[],[],[]
3,แทงหวย หวยออนไลน์ ลอตเตอรี่,True,"[ลอตเตอรี่, แทงหวย, หวย]",False,[],[],[]
4,casino slot bet jackpot,True,"[jackpot, bet, casino]",False,[],[],[]
5,p g s l o t แจกเครดิตฟรี,True,[เครดิตฟรี],False,[],[],[]
6,sl0t แตกหนักมาก,True,[sl0t],False,[],[],[]
7,c a s i n o online,False,[],False,[],[],[]
8,บ า ค า ร่า เล่นง่าย,False,[],False,[],[],[]
9,เ ว็ บ พ นั น ดี ที่ สุด,False,[],False,[],[],[]


In [ ]:
# @title 4. โหลดโมเดล Looloo NER (Smart Auto-Auth)
!pip install --upgrade transformers huggingface_hub

import torch
import sys
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from huggingface_hub import login
from IPython.display import display, HTML, clear_output
from getpass import getpass

LOOLOO_MODEL_ID = "loolootech/no-name-ner-th"

def _create_pipeline(token_val):
    use_cpu = not torch.cuda.is_available()
    tokenizer = AutoTokenizer.from_pretrained(LOOLOO_MODEL_ID, token=token_val)
    model = AutoModelForTokenClassification.from_pretrained(LOOLOO_MODEL_ID, token=token_val)

    return pipeline(
        "token-classification",
        model=model,
        tokenizer=tokenizer,
        aggregation_strategy="simple",
        device=-1 if use_cpu else 0
    )

def load_looloo_ner_smart():
    print(f"⏳ กำลังพยายามโหลดโมเดล: {LOOLOO_MODEL_ID} ...")
    try:
        ner_pipeline = _create_pipeline(token_val=True)
        print("✅ โหลดโมเดลสำเร็จทันที! (ใช้ Cached Token)")
        return ner_pipeline
    except Exception as e:
        error_msg = str(e)
        if "401" not in error_msg and "gated" not in error_msg and "token" not in error_msg:
            print(f"❌ Error: {e}")
            return None

        clear_output()
        display(HTML(f'''
        <div style="background-color: #ffebee; padding: 15px; border-radius: 5px; border: 1px solid #ffcdd2; margin-bottom: 10px;">
            <h3 style="margin-top:0;">🛑 ขั้นตอนที่ 1: กดอนุญาตที่หน้าเว็บ</h3>
            <p>โมเดลนี้จำกัดการเข้าถึง คุณต้องไปกดปุ่ม <b>"Agree and access repository"</b> ที่ลิงก์ด้านล่าง</p>
            <a href="https://huggingface.co/{LOOLOO_MODEL_ID}" target="_blank" style="font-size: 16px; font-weight: bold; color: #d32f2f;">👉 คลิกที่นี่เพื่อไปหน้า Agree</a>
        </div>
        '''))
        new_token = getpass("วาง Token (Read) จาก HF แล้วกด Enter: ")
        try:
            login(token=new_token)
            return _create_pipeline(token_val=new_token)
        except:
            return None

looloo_ner_pipe = load_looloo_ner_smart()

วาง Token (Read) จาก HF แล้วกด Enter: ··········


tokenizer_config.json:   0%|          | 0.00/145k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def apply_looloo_ner(text):
    if looloo_ner_pipe is None: return text, []
    try: results = looloo_ner_pipe(text)
    except: return text, []

    masked_text = text
    sorted_ents = sorted(results, key=lambda x: x['start'], reverse=True)
    entities_found = []

    for entity in sorted_ents:
        start, end = entity['start'], entity['end']
        label = entity['entity_group'].upper()

        word = text[start:end]
        entities_found.append((word, label))

        # มาตรฐานการจัดกลุ่ม Tag
        if label in ['PERSON', 'PER']:
            tag = "PERSON"
        elif label in ['LOCATION', 'LOC', 'ADDRESS', 'ADDR']:
            tag = "ADDRESS"
        elif label in ['ORGANIZATION', 'ORG']:
            tag = "ORG"
        elif label in ['PHONE', 'TEL', 'MOBILE']:
            tag = "PHONE"
        elif label in ['EMAIL']:
            tag = "EMAIL"
        elif label in ['NATIONAL_ID', 'ID_CARD', 'ID']:
            tag = "NATIONAL_ID"
        else:
            tag = label # Tag อื่นๆ นอกเหนือจากที่ระบุ

        masked_text = masked_text[:start] + f"[{tag}]" + masked_text[end:]

    return masked_text, entities_found

def apply_looloo_raw(text):
    """Mask แบบดิบๆ ตามโมเดล Looloo สำหรับแสดงผลในคอลัมน์ RAW"""
    if looloo_ner_pipe is None: return text
    try: results = looloo_ner_pipe(text)
    except: return text

    sorted_ents = sorted(results, key=lambda x: x['start'], reverse=True)
    masked_text = text
    for ent in sorted_ents:
        lbl = ent['entity_group'].upper()

        if lbl in ['PERSON', 'PER']: tag = "PERSON"
        elif lbl in ['LOCATION', 'LOC', 'ADDRESS', 'ADDR']: tag = "ADDRESS"
        elif lbl in ['ORGANIZATION', 'ORG']: tag = "ORG"
        elif lbl in ['PHONE', 'TEL']: tag = "PHONE"
        elif lbl in ['EMAIL']: tag = "EMAIL"
        elif lbl in ['NATIONAL_ID']: tag = "NATIONAL_ID"
        else: tag = lbl

        masked_text = masked_text[:ent['start']] + f"[{tag}]" + masked_text[ent['end']:]
    return masked_text

In [ ]:
# @title 3. โหลดโมเดล NER (Looloo & PyThaiNLP)
!pip install --upgrade transformers huggingface_hub python-crfsuite

import torch
import sys
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from huggingface_hub import login
from IPython.display import display, HTML, clear_output
from getpass import getpass
from pythainlp.tag import NER

# ========================================================
# PART A: Looloo NER Setup (Smart Auto-Auth)
# ========================================================
LOOLOO_MODEL_ID = "loolootech/no-name-ner-th"

def _create_pipeline(token_val):
    use_cpu = not torch.cuda.is_available()
    tokenizer = AutoTokenizer.from_pretrained(LOOLOO_MODEL_ID, token=token_val)
    model = AutoModelForTokenClassification.from_pretrained(LOOLOO_MODEL_ID, token=token_val)

    return pipeline(
        "token-classification",
        model=model,
        tokenizer=tokenizer,
        aggregation_strategy="simple",
        device=-1 if use_cpu else 0
    )

def load_looloo_ner_smart():
    print(f"⏳ กำลังพยายามโหลดโมเดล: {LOOLOO_MODEL_ID} ...")
    try:
        ner_pipeline = _create_pipeline(token_val=True)
        print("✅ โหลดโมเดลสำเร็จทันที! (ใช้ Cached Token)")
        return ner_pipeline
    except Exception as e:
        error_msg = str(e)
        if "401" not in error_msg and "gated" not in error_msg and "token" not in error_msg:
            print(f"❌ Error: {e}")
            return None

        clear_output()
        display(HTML(f'''
        <div style="background-color: #ffebee; padding: 15px; border-radius: 5px; border: 1px solid #ffcdd2; margin-bottom: 10px;">
            <h3 style="margin-top:0;">🛑 ขั้นตอนที่ 1: กดอนุญาตที่หน้าเว็บ</h3>
            <p>โมเดลนี้จำกัดการเข้าถึง คุณต้องไปกดปุ่ม <b>"Agree and access repository"</b> ที่ลิงก์ด้านล่าง</p>
            <a href="https://huggingface.co/{LOOLOO_MODEL_ID}" target="_blank" style="font-size: 16px; font-weight: bold; color: #d32f2f;">👉 คลิกที่นี่เพื่อไปหน้า Agree</a>
        </div>
        '''))
        new_token = getpass("วาง Token (Read) จาก HF แล้วกด Enter: ")
        try:
            login(token=new_token)
            return _create_pipeline(token_val=new_token)
        except:
            return None

looloo_ner_pipe = load_looloo_ner_smart()

# ========================================================
# PART B: PyThaiNLP NER Setup
# ========================================================
print("\n⏳ กำลังโหลดโมเดล PyThaiNLP (thainer)...")
try:
    pythai_ner_engine = NER("thainer")
except:
    print("⚠️ Missing python-crfsuite, installing...")
    os.system("pip install python-crfsuite")
    pythai_ner_engine = NER("thainer")

print("✅ PyThaiNLP NER Loaded Successfully.")
print("🚀 All NER Models Ready!")

  Using cached huggingface_hub-1.3.1-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.2 MB/s eta 0:00:00
⏳ กำลังพยายามโหลดโมเดล: loolootech/no-name-ner-th ...


Device set to use cpu


✅ โหลดโมเดลสำเร็จทันที! (ใช้ Cached Token)

⏳ กำลังโหลดโมเดล PyThaiNLP (thainer)...
Corpus: thainer-1.4
- Downloading: thainer-1.4 1.4


  0%|          | 0/1872468 [00:00<?, ?it/s]

✅ PyThaiNLP NER Loaded Successfully.
🚀 All NER Models Ready!


In [ ]:
# @title 4. ระบบรวมและทำความสะอาดผล NER (Unified NER Post-Processing)

# ============================================================================
# 1) Standard Tag Mapping
# ============================================================================
STD_TAGS = {
    "PERSON": {"PERSON","PER","PSN","B-PER","I-PER","B-PERSON","I-PERSON"},
    "PHONE": {"PHONE","TEL","MOBILE","CONTACT","B-PHONE","I-PHONE"},
    "EMAIL": {"EMAIL","MAIL","B-EMAIL","I-EMAIL"},
    "ADDRESS": {
        "ADDRESS","ADDR","LOCATION","LOC","PLACE","GPE",
        "B-ADDRESS","I-ADDRESS","B-LOCATION","I-LOCATION",
        "B-LOC","I-LOC","B-GPE","I-GPE"
    },
    "DATE": {"DATE","TIME","B-DATE","I-DATE","B-TIME","I-TIME"},
    "NATIONAL_ID": {"NATIONAL_ID","ID","IDCARD","ID_CARD","B-NATIONAL_ID","I-NATIONAL_ID"},
    "ORGANIZATION": {"ORG","ORGANIZATION","ORGNAME","B-ORG","I-ORG","B-ORGANIZATION","I-ORGANIZATION"},
}

# ============================================================================
# 2) Business Rule: Tag ที่อนุญาตเท่านั้น
# ============================================================================
ALLOWED_TAGS = {
    "PERSON",
    "PHONE",
    "EMAIL",
    "DATE",
    "ADDRESS",
    "NATIONAL_ID",
    "ORGANIZATION"
}

# ============================================================================
# 3) Tag Normalizer
# ============================================================================
def normalize_tag(tag: str) -> str:
    if not tag:
        return tag
    tag = tag.upper()
    for std, variants in STD_TAGS.items():
        if tag in variants:
            return std
    return tag

# ============================================================================
# 4) Unified Entity Schema
# ============================================================================
def make_entity(text, tag, start=None, end=None, source=None):
    return {
        "text": text,
        "tag": normalize_tag(tag),
        "start": start,
        "end": end,
        "source": source
    }

# ============================================================================
# 5) Shared Cleaner (ใช้กับทุกโมเดล)
# ============================================================================
def clean_entities(entities):
    """
    - เก็บเฉพาะ tag ที่อยู่ใน ALLOWED_TAGS
    - ลบ text สั้น/ว่าง
    - ลบ entity ซ้ำติดกัน
    """
    cleaned = []
    for e in entities:
        tag = e["tag"]
        text = e.get("text","")

        if tag not in ALLOWED_TAGS:
            continue
        if not text or len(text.strip()) < 2:
            continue

        # กันซ้ำ
        if cleaned:
            prev = cleaned[-1]
            if prev["tag"] == tag and prev["text"] == text:
                continue

        cleaned.append(e)

    return cleaned

# ============================================================================
# 6) Looloo Specific Cleaner (แก้ span ผี + span ซ้อน)
# ============================================================================
def clean_looloo_raw(raw_entities):
    tmp = []
    for e in raw_entities:
        start, end = e.get("start"), e.get("end")
        word = e.get("word","")
        if not word or start is None or end is None:
            continue
        if (end - start) < 2:
            continue
        tmp.append(e)

    # sort: start asc, length desc
    tmp = sorted(tmp, key=lambda x: (x["start"], -(x["end"] - x["start"])))

    cleaned = []
    last_end = -1
    for e in tmp:
        if e["start"] >= last_end:
            cleaned.append(e)
            last_end = e["end"]
        else:
            continue

    return cleaned

# ============================================================================
# 7) Processors
# ============================================================================
# --- 7.1 Looloo ---
def process_looloo(text, raw_results):
    raw_results = clean_looloo_raw(raw_results)

    entities = []
    for ent in raw_results:
        start = ent["start"]
        end = ent["end"]
        raw_tag = ent.get("entity_group") or ent.get("entity")
        word = text[start:end]

        entities.append(make_entity(word, raw_tag, start, end, "looloo"))

    entities = clean_entities(entities)
    return entities

# --- 7.2 PyThaiNLP ---
def process_pythainlp(tags_list):
    entities = []
    buffer_words = []
    buffer_tag = None

    def flush():
        nonlocal buffer_words, buffer_tag
        if not buffer_words:
            return
        entities.append(make_entity("".join(buffer_words), buffer_tag, source="pythainlp"))
        buffer_words = []
        buffer_tag = None

    for word, tag in tags_list:
        if tag.startswith("B-"):
            flush()
            buffer_words = [word]
            buffer_tag = tag[2:]
        elif tag.startswith("I-") and buffer_tag == tag[2:]:
            buffer_words.append(word)
        else:
            flush()

    flush()

    entities = clean_entities(entities)
    return entities

# --- 7.3 Generic ---
def process_generic(results):
    entities = []
    for ent in results:
        entities.append(make_entity(
            ent.get("text"),
            ent.get("label") or ent.get("tag"),
            ent.get("start"),
            ent.get("end"),
            "generic"
        ))
    entities = clean_entities(entities)
    return entities

print("✅ Unified NER Post-Processing Ready (Clean Looloo + PyThaiNLP)")


✅ Unified NER Post-Processing Ready (Clean Looloo + PyThaiNLP)


In [ ]:
print("---- Looloo Test ----")
print(looloo_ner_pipe("นายสมชาย ใจดี ติดต่อที่ 0812345678 วันที่ 1 มกราคม 2566"))

print("\n---- PyThaiNLP Test ----")
print(pythai_ner_engine.tag("นายสมชาย ใจดี ติดต่อที่ 0812345678 วันที่ 1 มกราคม 2566"))

---- Looloo Test ----
[{'entity_group': 'PERSON', 'score': np.float32(0.9999697), 'word': '', 'start': 0, 'end': 1}, {'entity_group': 'PERSON', 'score': np.float32(0.999969), 'word': 'นายสมชาย', 'start': 0, 'end': 8}, {'entity_group': 'PHONE', 'score': np.float32(0.99981683), 'word': '0812345678', 'start': 23, 'end': 34}, {'entity_group': 'DATE', 'score': np.float32(0.99072355), 'word': '1 มกราคม', 'start': 41, 'end': 50}]

---- PyThaiNLP Test ----
[('นาย', 'B-PERSON'), ('สม', 'I-PERSON'), ('ชาย', 'I-PERSON'), (' ', 'I-PERSON'), ('ใจดี', 'I-PERSON'), (' ', 'O'), ('ติดต่อ', 'O'), ('ที่', 'O'), (' ', 'O'), ('0812345678', 'O'), (' ', 'O'), ('วันที่', 'O'), (' ', 'O'), ('1', 'B-DATE'), (' ', 'I-DATE'), ('มกราคม', 'I-DATE'), (' ', 'I-DATE'), ('2566', 'I-DATE')]


In [ ]:
# @title 5. เปรียบการ Mask ของ NER

import pandas as pd

# ============================================================================
# Helper: Mask text by entities (use start/end)
# ============================================================================
def mask_text_with_entities(text, entities):
    ents = [e for e in entities if e.get("start") is not None and e.get("end") is not None]
    ents = sorted(ents, key=lambda x: x["start"], reverse=True)

    masked = text
    for e in ents:
        s, e_end = e["start"], e["end"]
        tag = e["tag"]
        masked = masked[:s] + f"[{tag}]" + masked[e_end:]

    return masked

# ============================================================================
# Helper: Mask PyThaiNLP result (no offset → replace text heuristically)
# ============================================================================
def mask_text_pythai(text, entities):
    out = text
    for e in entities:
        if e["text"]:
            out = out.replace(e["text"], f"[{e['tag']}]", 1)
    return out

# ============================================================================
# Main Runner
# ============================================================================
def run_ner_and_build_df(text_list):
    rows = []

    for text in text_list:
        # ---------------- Looloo ----------------
        if looloo_ner_pipe is not None:
            raw_looloo = looloo_ner_pipe(text)
            looloo_entities = process_looloo(text, raw_looloo)
            looloo_mask = mask_text_with_entities(text, looloo_entities)
        else:
            looloo_mask = text

        # ---------------- PyThaiNLP ----------------
        if pythai_ner_engine is not None:
            raw_pythai = pythai_ner_engine.tag(text)   # ✅ จุดสำคัญ
            pythai_entities = process_pythainlp(raw_pythai)
            pythai_mask = mask_text_pythai(text, pythai_entities)
        else:
            pythai_mask = text

        rows.append({
            "text": text,
            "looloo_mask": looloo_mask,
            "pythai_mask": pythai_mask
        })

    return pd.DataFrame(rows)

# ============================================================================
# Test
# ============================================================================
test_texts = [

    # ===== PERSON =====
    "นายสมชาย ใจดี เป็นผู้ยื่นคำร้อง",
    "นางสาวสมหญิง รัตนา ลงนามในเอกสารนี้",

    # ===== PHONE =====
    "ติดต่อที่หมายเลข 0812345678 หรือ 02-123-4567",

    # ===== EMAIL =====
    "สามารถส่งเอกสารมาที่ test.user@gmail.com ภายในวันนี้",

    # ===== ADDRESS / LOCATION =====
    "ที่อยู่ปัจจุบันคือ 169 ถนนลงหาดบางแสน จังหวัดชลบุรี",
    "สถานที่จัดงานคือ กรุงเทพ",

    # ===== DATE / TIME =====
    "ยื่นเอกสารวันที่ 1 มกราคม 2566",
    "ประชุมเวลา 13:30 น. วันที่ 12/05/2024",

    # ===== NATIONAL_ID =====
    "เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนยันตัวตน",

    # ===== ORGANIZATION =====
    "โรงพยาบาลศิริราช เป็นหน่วยงานของรัฐ",
    "บริษัท ปตท. จำกัด (มหาชน) เป็นรัฐวิสาหกิจ",

    # ===== MIX: รวมหลาย tag ในประโยคเดียว =====
    "นายสมชาย ใจดี เลขบัตรประชาชน 1234567890123 ติดต่อ 0812345678 วันที่ 1 มกราคม 2566",
    "ผู้ป่วย HN 123456 รักษาที่ โรงพยาบาลศิริราช ค่าใช้จ่าย 5000 บาท",
    "ส่งอีเมลไปที่ test.user@gmail.com เพื่อแจ้งผลในวันที่ 12/05/2024",
    "บริษัท ปตท. จำกัด จัดงาน วันเด็กแห่งชาติ ที่ กรุงเทพ",
]


df = run_ner_and_build_df(test_texts)
df


,text,looloo_mask,pythai_mask
0,นายสมชาย ใจดี เป็นผู้ยื่นคำร้อง,[PERSON] เป็นผู้ยื่นคำร้อง,[PERSON] เป็นผู้ยื่นคำร้อง
1,นางสาวสมหญิง รัตนา ลงนามในเอกสารนี้,[PERSON] ลงนามในเอกสารนี้,[PERSON] ลงนามในเอกสารนี้
2,ติดต่อที่หมายเลข 0812345678 หรือ 02-123-4567,ติดต่อที่หมายเลข[PHONE] หรือ[PHONE],ติดต่อที่หมายเลข 0812345678 หรือ [PHONE]
3,สามารถส่งเอกสารมาที่ test.user@gmail.com ภายใน...,สามารถส่งเอกสารมาที่[EMAIL] ภายในวันนี้,สามารถส่งเอกสารมาที่ test.user@gmail.com ภายใน...
4,ที่อยู่ปัจจุบันคือ 169 ถนนลงหาดบางแสน จังหวัดช...,ที่อยู่ปัจจุบันคือ[ADDRESS],ที่อยู่ปัจจุบันคือ 169 [ADDRESS] [ADDRESS]
5,สถานที่จัดงานคือ กรุงเทพ,สถานที่จัดงานคือ [ADDRESS],สถานที่จัดงานคือ [ADDRESS]
6,ยื่นเอกสารวันที่ 1 มกราคม 2566,ยื่นเอกสารวันที่[DATE]66,ยื่นเอกสารวันที่ [DATE]
7,ประชุมเวลา 13:30 น. วันที่ 12/05/2024,ประชุมเวลา[ADDRESS]. วันที่[DATE],ประชุมเวลา [DATE] วันที่ 12/05/2024
8,เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนย...,เลขประจำตัวประชาชน 12[NATIONAL_ID] ใช้สำหรับยื...,เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนย...
9,โรงพยาบาลศิริราช เป็นหน่วยงานของรัฐ,[ADDRESS] เป็นหน่วยงานของรัฐ,[ORGANIZATION] เป็นหน่วยงานของรัฐ


In [ ]:
# @title 6. ระบบจัดรูปแบบข้อความและทำข้อมูลปลอม (Final Formatter & Anonymizer)

!pip install git+https://github.com/PyThaiNLP/Thai-Data-Privacy -q

import random
import re

# ============================================================================
# Thai Prefixes
# ============================================================================
TH_PREFIXES = [
    "นาย","นาง","นางสาว","น.ส.","นส.","คุณ","ดร.","ศ.","รศ.","ผศ.",
    "พล.อ.","พล.ท.","พล.ต.","พล.ร.อ.","พล.อ.อ.","พันเอก","พันโท","พันตรี",
    "นพ.","พญ.","คุณหญิง","ท่าน","เลขา"
]

# ============================================================================
# ThaiDP Generator Wrapper
# ============================================================================
class ThaiDPPerfectGenerator:
    """คลาสสำหรับสร้างข้อมูลสมมติ (Replacement)"""

    def __init__(self):
        try:
            from thaidp import gen
            self.gen = gen
            print("✅ ThaiDP gen loaded")
        except Exception as e:
            print("⚠️ ThaiDP not available, fallback mode:", e)
            self.gen = None

        self.cache = {}
        self.provinces = [
            "กรุงเทพมหานคร", "เชียงใหม่", "ขอนแก่น",
            "สงขลา", "ภูเก็ต", "ชลบุรี"
        ]

    def _clean_prefix(self, text):
        sorted_pfx = sorted(TH_PREFIXES, key=len, reverse=True)
        pattern = r'^(' + '|'.join(map(re.escape, sorted_pfx)) + r')\s*'
        return re.sub(pattern, '', text or "")

    def get_replacement(self, label, text):
        label = (label or "").upper()
        text = text or ""
        pure_name = self._clean_prefix(text).strip()

        # PERSON
        if label == "PERSON":
            if not pure_name:
                return text

            if pure_name not in self.cache:
                try:
                    if self.gen:
                        full_name = self.gen.gen_name()
                        parts = full_name.split()
                        first = parts[0]
                        last = parts[-1] if len(parts) > 1 else "ใจดี"

                        if " " in pure_name:
                            self.cache[pure_name] = f"{first} {last}"
                        else:
                            self.cache[pure_name] = first
                    else:
                        raise Exception("ThaiDP not loaded")
                except:
                    self.cache[pure_name] = "สมชาย ใจดี"

            return self.cache[pure_name]

        elif label == "ADDRESS":
            return f"เลขที่ {random.randint(1,500)} ถ.สุขุมวิท {random.choice(self.provinces)}"

        elif label == "PHONE":
            try:
                if self.gen:
                    return self.gen.gen_phone()
            except:
                pass
            return f"08{random.randint(1,9)}-{random.randint(100,999)}-{random.randint(1000,9999)}"

        elif label == "EMAIL":
            try:
                if self.gen:
                    return self.gen.gen_email()
            except:
                pass
            return "anon@example.com"

        elif label == "NATIONAL_ID":
            try:
                if self.gen:
                    return self.gen.gen_citizen_id()
            except:
                pass
            return "1-1234-56789-01-2"

        elif label == "ORGANIZATION":
            return "บริษัท ตัวอย่าง จำกัด"

        elif label == "DATE":
            return "1 มกราคม 2566"

        elif label == "HOSPITAL_IDS":
            return f"HN{random.randint(100000,999999)}"

        return f"[{label}]"

real_data_gen = ThaiDPPerfectGenerator()

# ============================================================================
# Formatter
# ============================================================================
def format_with_entities(text, entities, mode="mask"):
    """
    mode = "mask" | "anon"
    รองรับ:
    - entity ที่มี start/end (Looloo)
    - entity ที่ไม่มี start/end (PyThaiNLP)
    """
    if not entities:
        return text

    out = text

    # ------------------------------------------------------------------
    # 1) แยก entity เป็น 2 กลุ่ม
    # ------------------------------------------------------------------
    with_offset = [e for e in entities if e.get("start") is not None]
    without_offset = [e for e in entities if e.get("start") is None]

    # ------------------------------------------------------------------
    # 2) จัดการพวกมี offset (Looloo) ก่อน
    # ------------------------------------------------------------------
    with_offset = sorted(with_offset, key=lambda x: x["start"], reverse=True)

    for e in with_offset:
        s, e_end = e["start"], e["end"]
        tag = e["tag"]
        chunk = out[s:e_end]

        # extract prefix
        prefix = ""
        for p in sorted(TH_PREFIXES, key=len, reverse=True):
            if chunk.startswith(p):
                prefix = p
                chunk = chunk[len(p):]
                break

        if mode == "mask":
            repl = prefix + f"[{tag}]"
        else:
            fake = real_data_gen.get_replacement(tag, chunk.strip())
            repl = prefix + fake

        out = out[:s] + repl + out[e_end:]

    # ------------------------------------------------------------------
    # 3) จัดการพวกไม่มี offset (PyThaiNLP)
    # ------------------------------------------------------------------
    for e in without_offset:
        tag = e["tag"]
        chunk = e.get("text","")
        if not chunk or chunk not in out:
            continue

        if mode == "mask":
            repl = f"[{tag}]"
        else:
            repl = real_data_gen.get_replacement(tag, chunk.strip())

        out = out.replace(chunk, repl, 1)

    return out

print("✅ Cell 6 FIXED: Formatter supports both Looloo & PyThaiNLP")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.8/788.8 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
# @title 7. เปรียบเทียบการ Mask และ Anonymize (Looloo vs PyThaiNLP)


import pandas as pd

# ============================================================================
# Test Cases
# ============================================================================
test_texts = [
    "นายสมชาย ใจดี เป็นผู้ยื่นคำร้อง",
    "ติดต่อที่หมายเลข 0812345678 หรือ 02-123-4567",
    "สามารถส่งเอกสารมาที่ test.user@gmail.com ภายในวันนี้",
    "ที่อยู่ปัจจุบันคือ 169 ถนนลงหาดบางแสน จังหวัดชลบุรี",
    "ยื่นเอกสารวันที่ 1 มกราคม 2566",
    "เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนยันตัวตน",
    "โรงพยาบาลศิริราช เป็นหน่วยงานของรัฐ",
    "บริษัท ปตท. จำกัด (มหาชน) เป็นรัฐวิสาหกิจ",
    "นายสมชาย ใจดี เลขบัตรประชาชน 1234567890123 ติดต่อ 0812345678 วันที่ 1 มกราคม 2566",
    "ผู้ป่วย HN 123456 รักษาที่ โรงพยาบาลศิริราช"
]

# ============================================================================
# Runner
# ============================================================================
rows = []

for text in test_texts:
    # ---------------- Looloo ----------------
    if looloo_ner_pipe is not None:
        raw_looloo = looloo_ner_pipe(text)
        looloo_entities = process_looloo(text, raw_looloo)

        looloo_mask = format_with_entities(text, looloo_entities, mode="mask")
        looloo_result = format_with_entities(text, looloo_entities, mode="anon")
    else:
        looloo_mask = text
        looloo_result = text

    # ---------------- PyThaiNLP ----------------
    if pythai_ner_engine is not None:
        raw_pythai = pythai_ner_engine.tag(text)
        pythai_entities = process_pythainlp(raw_pythai)

        pythai_mask = format_with_entities(text, pythai_entities, mode="mask")
        pythai_result = format_with_entities(text, pythai_entities, mode="anon")
    else:
        pythai_mask = text
        pythai_result = text

    rows.append({
        "text": text,
        "looloo_mask": looloo_mask,
        "pythai_mask": pythai_mask,
        "looloo_result": looloo_result,
        "pythai_result": pythai_result
    })

df = pd.DataFrame(rows)
df


,text,looloo_mask,pythai_mask,looloo_result,pythai_result
0,นายสมชาย ใจดี เป็นผู้ยื่นคำร้อง,นาย[PERSON] เป็นผู้ยื่นคำร้อง,[PERSON] เป็นผู้ยื่นคำร้อง,นายอามีเน๊าะ ใจดี เป็นผู้ยื่นคำร้อง,อามีเน๊าะ ใจดี เป็นผู้ยื่นคำร้อง
1,ติดต่อที่หมายเลข 0812345678 หรือ 02-123-4567,ติดต่อที่หมายเลข[PHONE] หรือ[PHONE],ติดต่อที่หมายเลข 0812345678 หรือ [PHONE],ติดต่อที่หมายเลข086-809-5201 หรือ083-913-3382,ติดต่อที่หมายเลข 0812345678 หรือ 088-846-7034
2,สามารถส่งเอกสารมาที่ test.user@gmail.com ภายใน...,สามารถส่งเอกสารมาที่[EMAIL] ภายในวันนี้,สามารถส่งเอกสารมาที่ test.user@gmail.com ภายใน...,สามารถส่งเอกสารมาที่kgraham@example.net ภายในว...,สามารถส่งเอกสารมาที่ test.user@gmail.com ภายใน...
3,ที่อยู่ปัจจุบันคือ 169 ถนนลงหาดบางแสน จังหวัดช...,ที่อยู่ปัจจุบันคือ[ADDRESS],ที่อยู่ปัจจุบันคือ 169 [ADDRESS] [ADDRESS],ที่อยู่ปัจจุบันคือเลขที่ 286 ถ.สุขุมวิท ชลบุรี,ที่อยู่ปัจจุบันคือ 169 เลขที่ 12 ถ.สุขุมวิท กร...
4,ยื่นเอกสารวันที่ 1 มกราคม 2566,ยื่นเอกสารวันที่[DATE]66,ยื่นเอกสารวันที่ [DATE],ยื่นเอกสารวันที่1 มกราคม 256666,ยื่นเอกสารวันที่ 1 มกราคม 2566
5,เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนย...,เลขประจำตัวประชาชน 12[NATIONAL_ID] ใช้สำหรับยื...,เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนย...,เลขประจำตัวประชาชน 121-1234-56789-01-2 ใช้สำหร...,เลขประจำตัวประชาชน 1234567890123 ใช้สำหรับยืนย...
6,โรงพยาบาลศิริราช เป็นหน่วยงานของรัฐ,[ADDRESS] เป็นหน่วยงานของรัฐ,[ORGANIZATION] เป็นหน่วยงานของรัฐ,เลขที่ 190 ถ.สุขุมวิท สงขลา เป็นหน่วยงานของรัฐ,บริษัท ตัวอย่าง จำกัด เป็นหน่วยงานของรัฐ
7,บริษัท ปตท. จำกัด (มหาชน) เป็นรัฐวิสาหกิจ,[ADDRESS]) เป็นรัฐวิสาหกิจ,[ORGANIZATION] เป็นรัฐวิสาหกิจ,เลขที่ 388 ถ.สุขุมวิท กรุงเทพมหานคร) เป็นรัฐวิ...,บริษัท ตัวอย่าง จำกัด เป็นรัฐวิสาหกิจ
8,นายสมชาย ใจดี เลขบัตรประชาชน 1234567890123 ติด...,นาย[PERSON] เลขบัตรประชาชน 12[NATIONAL_ID] ติด...,[PERSON] 1234567890123 ติดต่อ 0812345678 วันที...,นายอามีเน๊าะ ใจดี เลขบัตรประชาชน 121-1234-5678...,พิมพ์รตา ใจดี 1234567890123 ติดต่อ 0812345678 ...
9,ผู้ป่วย HN 123456 รักษาที่ โรงพยาบาลศิริราช,ผู้ป่วย HN 123456 รักษาที่ [ADDRESS],ผู้ป่วย HN 123456 รักษาที่ [ORGANIZATION],ผู้ป่วย HN 123456 รักษาที่ เลขที่ 102 ถ.สุขุมว...,ผู้ป่วย HN 123456 รักษาที่ บริษัท ตัวอย่าง จำกัด


In [ ]:
# @title 🛠️ Setup UI
import ipywidgets as widgets
import pandas as pd
import json
import io
from IPython.display import display, clear_output

def run_ui():
    # ==============================
    # 1. Helper: Smart Text Column Finder
    # ==============================
    def find_text_column(datum):
        keys = list(datum.keys())
        candidates = ['text', 'content', 'body', 'sentence', 'message', 'msg', 'comment', 'review', 'description', 'detail']
        for c in candidates:
            for k in keys:
                if k.lower() == c: return k
        for k in keys:
            val = datum[k]
            if isinstance(val, str) and len(val) > 5:
                return k
        return keys[0] if keys else None

    # ==============================
    # 2. UI Components
    # ==============================
    header = widgets.HTML("<h2>🛡️ Thai PDPA & Cleaning Pipeline </h2>")

    # --- Regex & Filter ---
    lbl_regex = widgets.HTML("<b>1. Regex & Content Filter</b>")
    chk_clean_header = widgets.Checkbox(value=True, description='Clean Header/Footer')
    chk_clean_repeat = widgets.Checkbox(value=True, description='Remove Repetitive Lines')
    chk_filter_gambling = widgets.Checkbox(value=True, description='Detect Gambling')
    chk_filter_rude = widgets.Checkbox(value=True, description='Detect Rude Words')

    regex_grid = widgets.GridBox(
        children=[chk_clean_header, chk_filter_gambling, chk_clean_repeat, chk_filter_rude],
        layout=widgets.Layout(width='100%', grid_template_columns='50% 50%')
    )
    regex_container = widgets.VBox([lbl_regex, regex_grid], layout=widgets.Layout(margin='0 0 15px 0'))

    # --- NER & Action ---
    lbl_ner = widgets.HTML("<b>2. Privacy Model</b>")
    dropdown_model = widgets.Dropdown(
        options=['None', 'Looloo (Deep Learning)', 'PyThaiNLP (Fast)'],
        value='Looloo (Deep Learning)',
        description='Model:',
        layout=widgets.Layout(width='95%')
    )
    dropdown_action = widgets.Dropdown(
        options=['Masking (Replace with [TAG])', 'Anonymization (Replace with Fake Data)'],
        value='Masking (Replace with [TAG])',
        description='Action:',
        layout=widgets.Layout(width='95%')
    )

    # --- Entity Selector ---
    lbl_tags = widgets.HTML("<b>3. Select Entities</b>")
    tags_options = ['PERSON', 'PHONE', 'EMAIL', 'ADDRESS', 'DATE', 'NATIONAL_ID', 'ORGANIZATION']
    selector_tags = widgets.SelectMultiple(
        options=tags_options,
        value=tags_options,
        rows=len(tags_options),
        layout=widgets.Layout(width='95%')
    )

    right_panel = widgets.VBox([
        lbl_ner, dropdown_model,
        widgets.HTML("<div style='margin-top:5px'></div>"),
        dropdown_action,
        widgets.HTML("<div style='margin-top:10px'></div>"),
        lbl_tags, selector_tags
    ])

    # --- Tabs ---
    input_text = widgets.Textarea(value='นายสมชาย โทร 081-234-5678 แจ้งเรื่องเว็บพนัน ufa888', placeholder='...', layout=widgets.Layout(width='100%', height='120px'))
    file_upload = widgets.FileUpload(accept='.json,.jsonl', multiple=False, description='Upload File')
    lbl_file_info = widgets.HTML("<i>*รองรับ .json/.jsonl</i>")

    input_tabs = widgets.Tab(children=[
        widgets.VBox([widgets.HTML("<b>Single Text:</b>"), input_text]),
        widgets.VBox([widgets.HTML("<b>File Upload:</b>"), file_upload, lbl_file_info])
    ])
    input_tabs.set_title(0, '📝 Text Input')
    input_tabs.set_title(1, '📂 File Upload')

    btn_run = widgets.Button(description='🚀 Run Pipeline', button_style='primary', layout=widgets.Layout(width='100%', height='50px', margin='15px 0px'), icon='play')
    out_log = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '8px', 'background_color': '#f9f9f9'})
    out_result = widgets.Output()

    # ==============================
    # 3. Logic
    # ==============================
    def process_item(text, selected_tags):
        flags = []
        # Regex
        if chk_clean_header.value: text = regex_processor.remove_header_footer(text)
        if chk_clean_repeat.value: text = regex_processor.remove_repetitive(text)
        if chk_filter_gambling.value and regex_processor.check_gambling(text)[0]: flags.append("Gambling")
        if chk_filter_rude.value and regex_processor.check_rude_words(text)[0]: flags.append("Rude")

        # NER
        model = dropdown_model.value
        entities = []
        if "Looloo" in model and 'looloo_ner_pipe' in globals():
            entities = process_looloo(text, looloo_ner_pipe(text))
        elif "PyThaiNLP" in model and 'pythai_ner_engine' in globals():
            entities = process_pythainlp(pythai_ner_engine.tag(text))

        # Action
        filtered_ents = [e for e in entities if e['tag'] in selected_tags]
        mode = "mask" if "Masking" in dropdown_action.value else "anon"

        final_text = text
        if model != 'None':
            final_text = format_with_entities(text, filtered_ents, mode=mode)
            if filtered_ents:
                flags.append("Masked" if mode == "mask" else "Anonymized")

        flag_str = " | ".join(flags) if flags else "Clean"
        return final_text, flag_str, len(filtered_ents)

    def on_click_run(b):
        out_log.clear_output()
        out_result.clear_output()
        sel_tags = selector_tags.value

        with out_log:
            # Single Text
            if input_tabs.selected_index == 0:
                txt = input_text.value
                if not txt: print("❌ Empty text"); return
                print("⏳ Processing...")
                res, flag, cnt = process_item(txt, sel_tags)
                print(f"✅ Result: {flag} (Protected Count: {cnt})")
                with out_result: display(widgets.Textarea(value=res, layout=widgets.Layout(width='100%', height='120px')))

            # File Upload
            else:
                if not file_upload.value: print("❌ No file uploaded"); return
                fname = list(file_upload.value.keys())[0]
                content = file_upload.value[fname]['content']
                print(f"⏳ Reading file: {fname} ...")

                data = []
                try:
                    decoded = content.decode('utf-8')
                    try: # JSONL
                        lines = decoded.strip().splitlines()
                        if lines[0].strip().startswith('{'): data = [json.loads(line) for line in lines[:50] if line.strip()]
                        else: raise ValueError
                    except: # JSON
                        data = json.loads(decoded)
                        if isinstance(data, list): data = data[:50]
                except: print("❌ Error parsing file"); return

                if not data: print("❌ Empty data"); return
                target_col = find_text_column(data[0])
                print(f"   -> Column: '{target_col}'")

                results = []
                for row in data:
                    raw_txt = str(row.get(target_col, ""))
                    proc_txt, flag, cnt = process_item(raw_txt, sel_tags)

                    # --- เปลี่ยนชื่อ Key ให้สื่อความหมายตรงนี้ ---
                    results.append({
                        'Original Input': raw_txt,
                        'Cleaned Result': proc_txt,
                        'Protected Count': cnt,  # เปลี่ยนจาก entities_found
                        'Status / Alerts': flag   # เปลี่ยนจาก flags
                    })

                print("✅ Processing Complete!")
                # แสดงผลตาราง
                df = pd.DataFrame(results)
                with out_result: display(df)

    btn_run.on_click(on_click_run)

    # Layout
    left_side = widgets.VBox([regex_container, input_tabs])
    ui = widgets.VBox([
        header,
        widgets.HBox([left_side, right_panel], layout=widgets.Layout(width='100%', justify_content='space-between')),
        widgets.HTML("<hr>"),
        btn_run,
        out_log,
        out_result
    ])
    display(ui)
run_ui()